In [327]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import matplotlib.pyplot as plt
import tensorflow

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from gensim.models import Word2Vec
from google.colab import drive
drive.mount('/content/drive')

import re
import pandas as pd
from bnlp import NLTKTokenizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [328]:
!pip install gensim

In [329]:
train_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/SC/New_shuffle.xlsx')
test_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/SC/bala 1.xlsx')
if set(['No','Comment', 'Tag']).issubset(train_data.columns):
  train_data = train_data.drop("No",axis=1)
# train_data
if set(['No','Comment','Tag']).issubset(test_data.columns):
  test_data = test_data.drop("No",axis=1)
# test_data

In [330]:
train_data.info() # Basic Information
print("\nShape: ",train_data.shape) # Shape of data
print("\nChecking for missing values")
train_data.isnull().sum()# Check for missing values
print("Count of sentiment wise values: \n",train_data.Tag.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6750 entries, 0 to 6749
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  6750 non-null   object
 1   Tag      6750 non-null   object
dtypes: object(2)
memory usage: 105.6+ KB

Shape:  (6750, 2)

Checking for missing values
Count of sentiment wise values: 
 Negative         1350
Very Positive    1350
Very Negative    1350
Positive         1350
Neutral          1350
Name: Tag, dtype: int64


In [ ]:
fig=px.histogram(train_data,
                x='Tag',
                title='Sentiment Count ',
                color_discrete_sequence=['green'])
fig.update_layout(bargap=0.1)
fig.show()
fig = px.pie(train_data, names='Tag',title="Sentiment Distribution")
fig.show()

In [332]:


# Manually define Bengali stop words
bengali_stop_words = {"এই", "এবং", "অথবা", "এইসব", "সে", "তার", "তারা", "তাদের"}

def bengali_data_cleaning(data):
    cleaned_data = []

    for i in range(len(data)):
        if isinstance(data[i], str):
            tweet = re.sub("#", "", data[i])
            tweet = re.sub(r'^https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
            html = re.compile(r'<.*?>')
            tweet = html.sub(r"", tweet)

            # Extracting symbols and characters
            tweet = re.sub(r'@\w+', "", tweet)
            tweet = re.sub(r'#\w+', "", tweet)
            tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)
            punctuation = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""
            tweet = re.sub('[^ঀ-৾\s]+', "", tweet)
            tweet = tweet.lower()

            # Tokenization
            tokenizer = NLTKTokenizer()
            tweet_tokens = tokenizer.word_tokenize(tweet)
            tweet = ' '.join(word for word in tweet_tokens if word not in bengali_stop_words)

            cleaned_data.append(tweet)
        else:
            cleaned_data.append("")

    return cleaned_data

#### Preprocessing and cleaning train data
texto_data = train_data['Comment']
cleaned_tx = bengali_data_cleaning(texto_data)

cleaned_df = pd.DataFrame({'Cleaned_Comment': cleaned_tx})
cleaned_df.index = train_data.index
train_data['Comment'] = cleaned_df['Cleaned_Comment']
train_data.head()

,Comment,Tag
0,খুটি তে বাধার নিয়ম ইংরেজ চালু করেছিল তা এখনো ...,Negative
1,যারা জনগণের প্রতি দায়বদ্ধ দায়িত্ববোধ অনেক বেশি,Very Positive
2,খেলার জন্যও মানুষ আপনাকে মনে রাখবে জনগণের জন্য...,Very Positive
3,বলে কি সাব্বির কেন বাংলাদেশ ক্রিকেটের প্রতি এট...,Very Negative
4,দিন দিন ব্যংক এ আমানত কারিদের টাকা র উপর লাভ ক...,Negative


In [333]:
texto_data = test_data['Comment']
cleaned_tx = bengali_data_cleaning(texto_data)

cleaned_df = pd.DataFrame({'Cleaned_Comment': cleaned_tx})
cleaned_df.index = test_data.index
test_data['Comment'] = cleaned_df['Cleaned_Comment']
test_data.head()

,Comment,Tag
0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন,Negative
1,অর্জন আমাদের গর্ব আমাদের অহংকার,Very Positive
2,তাহলে এ থেকে পাবলিক কী শিক্ষা নিতে পারে,Negative
3,সিটিসেলের নাম কি আর কইতাম আর কতই নাটক করবে,Very Negative
4,শেয়ার প্রতি আয় কমেছে শূন্য হয় নি এখনও সরকারী অ...,Negative


In [334]:
X_train = train_data.Comment
Y_train = train_data.Tag
Y_train.shape
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
Y_train = pd.DataFrame(Y_train,columns=['Tag'])
Y_train.head()

,Tag
0,0
1,4
2,4
3,3
4,0


In [335]:
test_data.head()
X_test = test_data.Comment
Y_test = test_data.Tag

In [336]:
X_test.head()
Y_test.head()

0         Negative
1    Very Positive
2         Negative
3    Very Negative
4         Negative
Name: Tag, dtype: object

In [337]:
Y_test = encoder.fit_transform(Y_test)
Y_test = pd.DataFrame(Y_test,columns=['Sentiment'])
Y_test.head()

,Sentiment
0,0
1,4
2,0
3,3
4,0


In [338]:
vectorizer = CountVectorizer(max_features= 3000, min_df = 3, max_df = 0.6)
# fit the model for training data
X_train_data = vectorizer.fit_transform(X_train)
X_train_data.shape

(6750, 1496)

In [339]:
X_test_data = vectorizer.fit_transform(X_test)
X_train_data.shape

(6750, 1496)

In [340]:
Y_train.shape

(6750, 1)

In [341]:
transformer = TfidfTransformer()
X_train_data = transformer.fit_transform(X_train_data).toarray()

print(X_train_data.shape)
#X_train_data[0]

(6750, 1496)


In [342]:
Y_train.head()

,Tag
0,0
1,4
2,4
3,3
4,0


In [343]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X_train_data, Y_train, test_size=0.33,random_state = 15)

In [344]:
model1_nb = MultinomialNB()
y_train_pred = model1_nb.fit(X1_train,y1_train.Tag).predict(X1_train)

y_pred = model1_nb.predict(X1_test)

# comparing actual response values  with predicted response values
from sklearn import metrics
print("Multinomial Naive Bayes model train accuracy(in %):", metrics.accuracy_score(y1_train, y_train_pred)*100)
print("Multinomial Naive Bayes model test accuracy(in %):", metrics.accuracy_score(y1_test, y_pred)*100)

Multinomial Naive Bayes model train accuracy(in %): 54.57762052189297
Multinomial Naive Bayes model test accuracy(in %): 26.61579892280072


# ** Deep Learning Model **

In [345]:
import tensorflow.keras.preprocessing.sequence as ps
from keras.constraints import max_norm
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional
from sklearn.preprocessing import OneHotEncoder

from keras.callbacks import EarlyStopping
from keras.layers import GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

In [346]:
print("X_train\n",X_train.head())
print("Y_train\n",Y_train.head())

print("X_test\n",X_test.head())
print("Y_test\n",Y_test.head())

X_train
 0    খুটি তে বাধার নিয়ম ইংরেজ চালু করেছিল তা এখনো ...
1       যারা জনগণের প্রতি দায়বদ্ধ দায়িত্ববোধ অনেক বেশি
2    খেলার জন্যও মানুষ আপনাকে মনে রাখবে জনগণের জন্য...
3    বলে কি সাব্বির কেন বাংলাদেশ ক্রিকেটের প্রতি এট...
4    দিন দিন ব্যংক এ আমানত কারিদের টাকা র উপর লাভ ক...
Name: Comment, dtype: object
Y_train
    Tag
0    0
1    4
2    4
3    3
4    0
X_test
 0              লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন
1                      অর্জন আমাদের গর্ব আমাদের অহংকার
2              তাহলে এ থেকে পাবলিক কী শিক্ষা নিতে পারে
3           সিটিসেলের নাম কি আর কইতাম আর কতই নাটক করবে
4    শেয়ার প্রতি আয় কমেছে শূন্য হয় নি এখনও সরকারী অ...
Name: Comment, dtype: object
Y_test
    Sentiment
0          0
1          4
2          0
3          3
4          0


In [347]:
tokenizer = Tokenizer(num_words=50000,
                      lower=True,
                      split=' ',
                      char_level=False,
                      oov_token='<UNK>',
                      document_count=0)

In [348]:
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(Y_train)
wordindex=tokenizer.word_index

In [349]:
tokenizer_config = tokenizer.get_config()
tokenizer_config.keys()

dict_keys(['num_words', 'filters', 'lower', 'split', 'char_level', 'oov_token', 'document_count', 'word_counts', 'word_docs', 'index_docs', 'index_word', 'word_index'])

In [350]:
max_length = 50
train_sequence = tokenizer.texts_to_sequences(X_train)
train_padding = ps.pad_sequences(train_sequence,maxlen=max_length,padding='post')

test_sequence = tokenizer.texts_to_sequences(X_test)
test_padding = ps.pad_sequences(test_sequence,maxlen=max_length,padding='post')

In [351]:
train_padding.shape
Y_train.shape

(6750, 1)

In [352]:
from sklearn.preprocessing import OneHotEncoder
y_train = OneHotEncoder().fit_transform(Y_train)
y_train.shape
y_test = OneHotEncoder().fit_transform(Y_test)
y_test.shape

(3480, 5)

In [ ]:
# Building the BASELINE MODEL
base_model = Sequential()
base_model.add(Embedding(50000,128,input_length=train_padding.shape[1]))
base_model.add(GlobalAveragePooling1D())
base_model.add(Dense(8,activation='relu'))
base_model.add(Dense(5,activation='softmax'))
base_model.summary()

base_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
y_train=np.array(y_train.toarray())
train_padding=np.array(train_padding)
history_base = base_model.fit(train_padding,y_train ,epochs=10, validation_split=0.2)

In [ ]:
plt.style.use('seaborn-darkgrid')
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

plt.figure(figsize=(15, 10))
plt.subplot(1, 2, 1)
plot_graphs(history_base, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history_base, 'loss')
plt.ylim(0, None)

In [ ]:
regularise = tensorflow.keras.regularizers.l2(0.001)

model_r = Sequential()
model_r.add(Embedding(50000,128,input_length=train_padding.shape[1]))
model_r.add(Dropout(0.5))
model_r.add(GlobalAveragePooling1D())
model_r.add(Dense(8,activation='relu',kernel_regularizer=regularise))
model_r.add(Dropout(0.5))
model_r.add(Dense(5,activation='softmax'))
model_r.summary()

In [ ]:
model_r.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history_r = model_r.fit(train_padding,y_train ,epochs=8, validation_split=0.2)

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(1, 2, 1)
plot_graphs(history_r, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history_r, 'loss')
plt.ylim(0, None)

In [360]:
score = model_r.evaluate(test_padding,y_test.toarray())
print("Testing Accuracy(%): ", score[1]*100)

109/109 [==============================] - 0s 2ms/step - loss: 1.5731 - accuracy: 0.2724
Testing Accuracy(%):  27.241379022598267
